# Objectif 1 : Constitution d'une base adresse à partir de fichiers DVF
- Adresse complète
- Code commune INSEE
- Coordonnées GPS de la commune
- Coorodnnées GPS de l'adresse
- Code IRIS (code quartier de l'INSEE) (DEPRECATED : malheureusement, l'API qui délivrait cette information n'est plus disponible)

# Objectif 2 : Retraitement de la base DVF à partir des décisions prises suite à la Data Exploration
- Suppression des biens immobiliers ne correspondant pas à l'objectif (immobilier neuf, dépendances, locaux industriels ou commerciaux)
- Suppression des outliers
- Suppression des colonnes inutiles
- Suppression des biens dont la transaction est répétée sur plusieurs lignes (cf. Data Exploration !)

## 1. Cleaning des adresses

In [31]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [32]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import json
import requests
import time

In [33]:
# Fichier(s) à traiter
year = '2020'             # Remplacer par "2021-s1" pour traiter les données DVF S1 2021
filename = 'databases/valeursfoncieres-' + year + '.txt'

In [34]:
# Chargement du jeu de données
df = pd.read_csv(filename, sep = "|", decimal = ",")
df.head()

C:\Users\olivier\anaconda\envs\Geo\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (14,18,23,24,26,28,41) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Code service CH,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,07/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1061.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,BT,NaN,85.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1115.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1940.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1148.0


In [35]:
df = pd.read_csv("databases/valeursfoncieres-" + year + ".txt", sep = "|", decimal = ",")
df.shape

C:\Users\olivier\anaconda\envs\Geo\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (14,18,23,24,26,28,41) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(2459560, 43)

In [36]:
print(df.columns)

Index(['Code service CH', 'Reference document', '1 Articles CGI',
       '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI',
       'No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain'],
      dtype='object')


## Cleaning des adresses et export

In [37]:
# Manipulations sur les codes départements et codes communes
df['Code postal'] = df['Code postal'].fillna(-1).astype(int).astype(str).replace('-1', np.nan)
df['Code departement'] = df['Code departement'].apply(lambda x: str(x).zfill(2) if len(str(x)) == 1 else x)
df['Code commune'] = df['Code commune'].apply(lambda x: str(x).zfill(3) if len(str(x)) < 3 else x)
df['Code commune INSEE'] = df['Code departement'].astype(str) + df['Code commune'].astype(str)

# Création d'un champ "Ville" pour interroger les coordonnées GPS lorsque le champ Adresse ne donne rien
df['Ville'] = df['Code postal'] + " " + df['Commune']

df.head()

,Code service CH,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,Code commune INSEE,Ville
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,07/01/2020,Vente,...,NaN,NaN,NaN,NaN,NaN,T,NaN,1061.0,01072,1250 CEYZERIAT
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,NaN,NaN,NaN,NaN,BT,NaN,85.0,01203,1290 LAIZ
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,NaN,NaN,NaN,NaN,T,NaN,1115.0,01203,1290 LAIZ
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,NaN,NaN,NaN,NaN,T,NaN,1940.0,01203,1290 LAIZ
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,NaN,NaN,NaN,NaN,T,NaN,1148.0,01203,1290 LAIZ


In [38]:
# Modification des communes avec arrondissements
df['Commune'] = df['Commune'].replace(['MARSEILLE 1ER', 'MARSEILLE 2EME', 'MARSEILLE 3EME', 'MARSEILLE 4EME',
                                           'MARSEILLE 5EME', 'MARSEILLE 6EME', 'MARSEILLE 7EME', 'MARSEILLE 8EME',
                                           'MARSEILLE 9EME', 'MARSEILLE 10EME', 'MARSEILLE 11EME', 'MARSEILLE 12EME',
                                           'MARSEILLE 13EME', 'MARSEILLE 14EME', 'MARSEILLE 15EME', 'MARSEILLE 16EME',
                                           'LYON 1ER', 'LYON 2EME', 'LYON 3EME', 'LYON 4EME', 'LYON 5EME',
                                           'LYON 6EME', 'LYON 7EME', 'LYON 8EME', 'LYON 9EME',
                                           'PARIS 01', 'PARIS 02', 'PARIS 03', 'PARIS 04', 'PARIS 05', 'PARIS 06',
                                           'PARIS 07', 'PARIS 08', 'PARIS 09', 'PARIS 10', 'PARIS 11', 'PARIS 12',
                                           'PARIS 13', 'PARIS 14', 'PARIS 15', 'PARIS 16', 'PARIS 17', 'PARIS 18',
                                           'PARIS 19', 'PARIS 20'],
                                          ['MARSEILLE', 'MARSEILLE', 'MARSEILLE', 'MARSEILLE',
                                          'MARSEILLE', 'MARSEILLE', 'MARSEILLE', 'MARSEILLE',
                                          'MARSEILLE', 'MARSEILLE', 'MARSEILLE', 'MARSEILLE',
                                          'MARSEILLE', 'MARSEILLE', 'MARSEILLE', 'MARSEILLE',
                                          'LYON', 'LYON', 'LYON', 'LYON', 'LYON',
                                          'LYON', 'LYON', 'LYON', 'LYON',
                                          'PARIS', 'PARIS', 'PARIS', 'PARIS', 'PARIS', 'PARIS',
                                          'PARIS', 'PARIS', 'PARIS', 'PARIS', 'PARIS', 'PARIS',
                                          'PARIS', 'PARIS', 'PARIS', 'PARIS', 'PARIS', 'PARIS',
                                          'PARIS', 'PARIS'])

In [39]:
# Ajout d'apostrophes après L et D pour une meilleure reconnaissance des noms de rues 
# et un meilleur matching lorsqu'on cherche à ajouter des données

df['Voie'] = df['Voie'].replace([' L ', ' D '], [' L\'', ' D\''], regex = True)

# Suppression des tirets des noms de ville
df['Ville'] = df['Ville'].replace('-', ' ', regex=True).astype(str)

In [40]:
# Fonction de remplacement des NaN par 0
def escapeNan(row):
    val = str(row)
    if val == "nan":
        val = " "
    else:
        val = val.replace('.0','') # if from float values
        val = val + " "
    return val

In [41]:
# Création du champ "addressName" qui sera utilisée dans la requête API pour récupérer les coordonnées GPS
df['Adresse'] = df.apply(lambda row: 
    escapeNan(row['No voie'])
    + escapeNan(row['Type de voie'])         
    + escapeNan(row['Voie'])
    + escapeNan(row['Code postal']).zfill(5) # pour respecter le format des codes postaux
    + str(row['Commune'])
    , axis = 1).str.strip()

In [42]:
# Extract des adresses
ad = df[['No voie', 'Type de voie', 'Voie', 'Code postal', 'Commune', 'Code departement', 'Code commune INSEE', 'Ville', 'Adresse']]
ad.shape

(2459560, 9)

In [43]:
# On supprime les adresses dupliquées pour éviter d'interroger les API plusieurs fois avec exactement la même adresse
ad = ad.drop_duplicates()
ad.shape

(1059387, 9)

### Export des adresses

In [44]:
ad.to_csv('./databases/inter/01A - export_dvf_adresses_' + year + '.csv')

## 2. Filtrage de la base DVF

In [45]:
## Ajout des bases externes simples
dep_reg = pd.read_csv("databases/departements-region.csv",sep=",")
communes_littorales = pd.read_csv("databases/communes_littorales_2019.csv", sep=";")

In [46]:
df.shape

(2459560, 46)

In [47]:
# On ne conserve que les biens de type "Appartement" et "Maison" et uniquement les mutations de type "Vente"
df = df[(df["Type local"] == "Appartement") | (df["Type local"] == "Maison")]
df = df[df["Nature mutation"] == "Vente"]

In [48]:
df.shape

(923067, 46)

In [49]:
df['Type local'].value_counts()

Maison         526507
Appartement    396560
Name: Type local, dtype: int64

In [50]:
# Colonnes inutiles
df.drop(["Code service CH","Reference document","1 Articles CGI",
              "2 Articles CGI","3 Articles CGI","4 Articles CGI",
              "5 Articles CGI"], axis = 1, inplace = True)

# Suppression des lignes doublons
df.drop_duplicates(subset = None, inplace = True)
df = df.drop_duplicates(subset = ["Date mutation","Valeur fonciere","Code postal"], keep = False)

df.shape

(610668, 39)

In [64]:
df['Surface terrain'] = df['Surface terrain'].fillna(0)

In [65]:
# Calcul du prix au m2 par bien immobilier
df["Prix m2"] = df["Valeur fonciere"] / df["Surface reelle bati"]
df["Paris"]= df["Commune"].apply(lambda x : 1 if x == "PARIS" else 0)

Paris = df[df["Paris"] == 1]
Autres_villes = df[df["Paris"] == 0]

In [66]:
# Avant suppression des outliers
print("Nb lignes Paris avant suppression des outliers : ", Paris.shape)
print("Nb lignes Autres Villes avant suppression des outliers : ", Autres_villes.shape)

Nb lignes Paris avant suppression des outliers :  (27358, 41)
Nb lignes Autres Villes avant suppression des outliers :  (583310, 41)


In [67]:
# Gestion des valeurs aberrantes Paris : les critères sont plus restrictifs sur Paris car plus de valeurs extrêmes
outliers_max_Paris = Paris["Valeur fonciere"].quantile(0.97)
outliers_min_Paris = Paris["Valeur fonciere"].quantile(0.05)
outliers_max_Paris_surface = Paris["Surface reelle bati"].quantile(0.99)
outliers_min_Paris_surface = Paris["Surface reelle bati"].quantile(0.01)
outliers_max_Paris_prix_m2= Paris["Prix m2"].quantile(0.98)
outliers_min_Paris_prix_m2 = Paris["Prix m2"].quantile(0.1)

Paris = Paris[(Paris["Valeur fonciere"] < outliers_max_Paris) & 
              (Paris["Valeur fonciere"] > outliers_min_Paris) &
              (Paris["Surface reelle bati"] < outliers_max_Paris_surface) &
              (Paris["Surface reelle bati"] > outliers_min_Paris_surface) &
              (Paris["Prix m2"] < outliers_max_Paris_prix_m2) &
              (Paris["Prix m2"] > outliers_min_Paris_prix_m2) ]

print(outliers_max_Paris)
print(outliers_min_Paris)
print(outliers_max_Paris_surface)
print(outliers_min_Paris_surface)
print(outliers_max_Paris_prix_m2)
print(outliers_min_Paris_prix_m2)

del outliers_max_Paris, outliers_min_Paris, outliers_max_Paris_surface, outliers_min_Paris_surface
del outliers_max_Paris_prix_m2, outliers_min_Paris_prix_m2

2023354.9999999958
109000.0
210.0
9.0
20769.23076923077
7222.222222222223


In [70]:
# Gestion des valeurs abberantes des villes autres que Paris
outliers_max_Autres_villes = Autres_villes["Valeur fonciere"].quantile(0.996)
outliers_min_Autres_villes = Autres_villes["Valeur fonciere"].quantile(0.05)
outliers_max_Autres_villes_surface = Autres_villes["Surface reelle bati"].quantile(0.998)
outliers_min_Autres_villes_surface = Autres_villes["Surface reelle bati"].quantile(0.0005)
outliers_max_Autres_villes_prix_m2 = Autres_villes["Prix m2"].quantile(0.998)
outliers_min_Autres_villes_prix_m2 = Autres_villes["Prix m2"].quantile(0.125)
outliers_max_Autres_villes_surface_terrain = Autres_villes["Surface terrain"].quantile(0.998)


Autres_villes = Autres_villes[(Autres_villes["Valeur fonciere"] < outliers_max_Autres_villes) & 
              (Autres_villes["Valeur fonciere"] > outliers_min_Autres_villes) &
              (Autres_villes["Surface reelle bati"] < outliers_max_Autres_villes_surface) &
              (Autres_villes["Surface reelle bati"] > outliers_min_Autres_villes_surface) &
              (Autres_villes["Prix m2"] < outliers_max_Autres_villes_prix_m2) &
              (Autres_villes["Prix m2"] > outliers_min_Autres_villes_prix_m2) &
              (Autres_villes["Surface terrain"] < outliers_max_Autres_villes_surface_terrain)]

print(outliers_max_Autres_villes)
print(outliers_min_Autres_villes)
print(outliers_max_Autres_villes_surface)
print(outliers_min_Autres_villes_surface)
print(outliers_max_Autres_villes_prix_m2)
print(outliers_min_Autres_villes_prix_m2)
print(outliers_max_Autres_villes_surface_terrain)

del outliers_max_Autres_villes, outliers_min_Autres_villes, outliers_max_Autres_villes_surface
del outliers_min_Autres_villes_surface, outliers_max_Autres_villes_prix_m2, outliers_min_Autres_villes_prix_m2
del outliers_max_Autres_villes_surface_terrain

1170559.799999965
42500.0
294.0
10.0
14500.684500000638
994.3181818181819
4022.1459999999497


In [71]:
# Après suppression des outliers
print("Nb lignes Paris après suppression des outliers : ", Paris.shape)
print("Nb lignes Autres Villes après suppression des outliers : ", Autres_villes.shape)

Nb lignes Paris après suppression des outliers :  (23096, 41)
Nb lignes Autres Villes après suppression des outliers :  (500391, 41)


In [72]:
df_clean = pd.concat([Paris, Autres_villes], axis = 0)
del Paris, Autres_villes

In [73]:
## Ajout des régions via le code département et suppresion du département Corse
dep_reg = dep_reg.rename(columns = {"num_dep":"Code departement"})
dico2 = {"Code departement":{"2A":"201","2B":"202"}}
dep_reg = dep_reg.replace(dico2)
dep_reg["Code departement"] = dep_reg["Code departement"].astype('int32')
df_clean = df_clean.replace(dico2)
df_clean["Code departement"] = df_clean["Code departement"].astype('int32')
df_clean = df_clean.merge(right = dep_reg , on = 'Code departement', how = 'left')
df_clean = df_clean[(df_clean["Code departement"] != 201) & (df_clean["Code departement"] != 202)]

In [74]:
## Ajout des communes possédant une surface maritime
communes_littorales = communes_littorales.drop_duplicates(keep ="first")
df_clean = df_clean.merge(right = communes_littorales , on = 'Commune', how = 'left')
dummies = pd.get_dummies(df_clean["Classement"])
df_clean = pd.concat([df_clean, dummies],axis =1)
df_clean["Mer"] = df_clean["Mer"] .astype("object")
dico3 = {"Mer":{"1":"OUI","0":"NON"}}
df_clean = df_clean.replace(dico3)
df_clean.drop("Classement", axis = 1, inplace = True)

del communes_littorales, dep_reg, dico2, dico3, dummies

In [75]:
df_clean.shape

(522587, 44)

In [76]:
df_clean.groupby(['Code departement', 'Type local'])['Prix m2'].mean().to_csv('databases/prix_m2_par_dep' + year + '.csv')

In [77]:
# Export de la base qui sera enrichie des codes GPS et IRIS dans le script #2C
df_clean.to_csv("./databases/inter/01B - dvf_sans_gps_iris_" + year + ".csv")